In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data.csv')
data.head()

,patient_id,age,gender,race,admission_date,diagnosis1,diagnosis2,diagnosis3,diagnosis4,diagnosis5,hcpcs,procedure1,procedure2,procedure3,procedure4,procedure5
0,P302,70,F,Unknown,12/14/2030,41071,51881,99672,78551,5856,81001,66,3607,45,40,3722
1,P430,62,F,Unknown,4/28/2021,41401,5849,4139,4148,V4582,82550,66,3607,3722,8853,45
2,P492,78,F,Unknown,7/7/2048,41011,42821,5185,4271,9971,82948,66,3606,3722,8853,45
3,P651,80,F,Unknown,2/18/2016,2848,42731,486,11284,5781,86901,4131,3893,4513,9904,8871
4,P679,68,F,Unknown,10/21/2013,9961,25040,40391,5856,41400,85014,3927,3895,3995,9904,3895


In [3]:
data_ICD = pd.read_csv('2017-GEM-DC\\2017_I9gem.txt', delim_whitespace=True, names=('Source',
                                                                          'Target','Flags'))
data_ICD.head()

,Source,Target,Flags
0,0010,A000,0
1,0011,A001,0
2,0019,A009,0
3,0020,A0100,10000
4,0021,A011,0


In [4]:
ICD9_Code = []
for i in ['diagnosis1','diagnosis2','diagnosis3','diagnosis4','diagnosis5']:
    ICD9_Code.append(data[i].tolist())
Code = [item for sublist in ICD9_Code for item in sublist]
x = np.array(Code) 
Code = np.unique(x) 
data1 = pd.DataFrame(Code, columns = ['ICD9_Code'])
data1.head()

,ICD9_Code
0,1101
1,1120
2,1125
3,11284
4,1146


In [5]:
mapped = data1.merge(data_ICD, left_on = 'ICD9_Code', right_on = 'Source')
mapped = mapped.drop(['Source', 'Flags'], axis=1)
for i in mapped.columns:
    mapped[i] = mapped[i].astype(str)

In [6]:
mapped['Target'] = mapped['Target'].apply(lambda x: x[:3] + '.' + x[3:] if len(x) > 3 else x)
mapped = mapped.groupby('ICD9_Code')['Target'].apply(','.join).reset_index()

In [7]:
for i in ['diagnosis1','diagnosis2','diagnosis3','diagnosis4','diagnosis5']:
    data=data.merge(mapped,
         left_on=i,
         right_on='ICD9_Code')
    a = i + "_ICD10CM"
    data = data.rename(columns={"Target":a})
    data = data.drop(columns=['ICD9_Code'])
data.head()

,patient_id,age,gender,race,admission_date,diagnosis1,diagnosis2,diagnosis3,diagnosis4,diagnosis5,...,procedure1,procedure2,procedure3,procedure4,procedure5,diagnosis1_ICD10CM,diagnosis2_ICD10CM,diagnosis3_ICD10CM,diagnosis4_ICD10CM,diagnosis5_ICD10CM
0,P302,70,F,Unknown,12/14/2030,41071,51881,99672,78551,5856,...,66,3607,45,40,3722,I21.4,"J96.00,J96.90","T82.817A,T82.827A,T82.837A,T82.847A,T82.855A,T...",R57.0,N18.6
1,P621,63,F,White,7/14/2020,4254,V421,V441,40391,5856,...,9339,9383,9375,9381,9336,"I42.5,I42.8",Z94.1,Z93.1,I12.0,N18.6
2,P460,71,F,Hispanic,4/7/2028,41011,4280,4260,78551,3481,...,3601,3606,9920,9960,9604,I21.09,I50.9,I44.2,R57.0,G93.1
3,P554,77,F,Black,6/16/2032,41401,4139,99672,79430,V4581,...,3601,3606,3722,8856,9920,I25.10,"I20.8,I20.9","T82.817A,T82.827A,T82.837A,T82.847A,T82.855A,T...",R94.30,Z95.1
4,P678,74,F,White,11/11/2018,41401,3963,4148,4019,V4581,...,3602,3607,3723,8856,8853,I25.10,I08.0,"I25.5,I25.89,I25.9","I10,I16.9",Z95.1


In [8]:
data = data[["patient_id", "age","gender", "race", "admission_date",
             "diagnosis1", "diagnosis1_ICD10CM",
             "diagnosis2", "diagnosis2_ICD10CM",
             "diagnosis3", "diagnosis3_ICD10CM",
             "diagnosis4", "diagnosis4_ICD10CM",
             "diagnosis5", "diagnosis5_ICD10CM",
             "hcpcs",
             "procedure1", 
             "procedure2", 
             "procedure3", 
             "procedure4", 
             "procedure5"]]